[This dataset](https://www.kaggle.com/datasets/patricklford/largest-companies-analysis-worldwide) provides around 10,000 records of the largest companies worldwide. For a job-seeker, Dividend Yield is a key metric to consider when evaluating a company. Dividend Yield is the ratio of the annual dividend per share to the current price per share. So if a job-seeker is offered shares as part of their compensation, they would want to know the Dividend Yield of the company.

In [1]:
import pandas as pd
from fuzzywuzzy import fuzz

/opt/miniconda3/envs/si618/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
company_df = pd.read_csv('Projectdata/Companies_ranked_by_Dividend_Yield.csv')

In [ ]:
company_df.head()

In [ ]:
job_df = pd.read_csv('Projectdata/job_company_pair.csv')
# randomly select 5 name from job_df
job_df['name'].sample(5)

In [ ]:
# create a new df that only contains company name in job_df
job_df_name = job_df['name'].unique()

In [ ]:
job_df_name = pd.DataFrame(job_df_name,columns=['company_name'])

In [ ]:
job_df_name.head(5)

Since the two dataset are from different sources, we will match the company names using following steps:
1. Remove special characters and spaces from the company names in both datasets.
2. Convert the company names to lowercase.
3. Use the `fuzzywuzzy` library to match the company names.
4. If the match score is above a certain threshold, we will consider the two company names to be the same. We will create a new column called `is_top` in the `job_df` dataset to store if there is a match in the `company_df` dataset.
5. If `is_top` is False, check if the company name in the `job_df` dataset is a substring of any company name in the `company_df` dataset and vice versa. If it is, we will consider the two company names to be the same and change `is_top` to True.
6. If `is_top` is True, we will merge the relevant columns from the `company_df` dataset to the `job_df` dataset.

In [ ]:
import re
def clean_company_name(name):
    """
    Clean company name by:
    1. Removing special characters and spaces
    2. Converting to lowercase
    """
    # Convert to string in case of non-string input
    name = str(name)
    
    # Remove special characters and convert to lowercase
    cleaned_name = re.sub(r'[^a-zA-Z0-9]', '', name).lower()
    
    return cleaned_name

In [ ]:
def find_longest_substring(s1, s2):
    """
    Find the longest consecutive substring between two strings
    """
    # Remove non-alphanumeric characters
    s1 = re.sub(r'[^a-z0-9]', '', s1)
    s2 = re.sub(r'[^a-z0-9]', '', s2)
    
    # Find all consecutive substrings of at least 6 characters
    for length in range(min(len(s1), len(s2)), 5, -1):
        for i in range(len(s1) - length + 1):
            substring = s1[i:i+length]
            if substring in s2:
                return substring
    
    return None

In [ ]:
def match_company_names(job_df, company_df, match_threshold=80):
    """
    Match company names between job and company datasets
    
    Parameters:
    - job_df: DataFrame with job listings
    - company_df: DataFrame with company information
    - match_threshold: Fuzzy matching threshold (default 80)
    
    Returns:
    - Updated job_df with new columns
    """
    # Create copies to avoid modifying original dataframes
    job_df = job_df.copy()
    company_df = company_df.copy()
    
    job_df['cleaned_company_name'] = job_df['company_name'].apply(clean_company_name)
    company_df['cleaned_company_name'] = company_df['Name'].apply(clean_company_name)
    
    # Create a dictionary of company names for faster lookup
    company_dict = {row['cleaned_company_name']: row for _, row in company_df.iterrows()}
    
    def find_best_match(job_company):
        # Exact match
        if job_company in company_dict:
            return True, company_dict[job_company]
        
        # Fuzzy and substring matching
        best_match = None
        best_score = 0
        
        for company_name, company_info in company_dict.items():
            # Fuzzy ratio match
            ratio = fuzz.ratio(job_company, company_name)
            
            # Substring match bonus
            substring_bonus = (job_company in company_name or company_name in job_company)
            
            # Calculate score with substring bonus
            score = ratio + (20 if substring_bonus else 0)
            
            if score > best_score and score >= match_threshold:
                best_score = score
                best_match = company_info
        
        return best_match is not None, best_match
    
    # Vectorized matching
    results = job_df['cleaned_company_name'].apply(find_best_match)
    
    # Update DataFrame
    job_df['is_top'] = results.apply(lambda x: x[0])
    
    # Merge additional company information for matched companies
    def merge_company_info(row, matched_result):
        if matched_result[0]:
            # Merge relevant columns from company_df
            company_info = matched_result[1]
            columns_to_merge = ['dividend_yield_ttm','price (GBP)']
            for col in columns_to_merge:
                if col in company_df.columns and col not in job_df.columns:
                    row[col] = company_info[col]
        return row
    
    # Apply merging
    job_df = job_df.apply(lambda row: merge_company_info(row, 
                                       results.loc[row.name]), 
                           axis=1)
    
    # Optional: Drop the temporary cleaning column
    
    return job_df

In [ ]:
matched_job_df = match_company_names(job_df_name, company_df)

In [ ]:
matched_job_df.sort_values('is_top', ascending=False).head(5)

In [ ]:
matched_job_df.to_csv('Projectdata/matched_companies.csv', index=False)

In [ ]:
job_company_df = pd.read_csv('Projectdata/job_company_pair.csv')

In [ ]:
job_company_df['cleaned_company_name'] = job_company_df['name'].apply(clean_company_name)

In [ ]:
job_company_df = pd.merge(job_company_df, matched_job_df, how='left', on='cleaned_company_name')

In [ ]:
job_company_df.head(5)

In [ ]:
job_company_df.drop(columns=['cleaned_company_name','name'], inplace=True)

In [ ]:
job_company_df.to_csv('Projectdata/matched_job_company_pair.csv', index=False)

In [2]:
job_df = pd.read_csv('Projectdata/cleaned_job_company_pair.csv')

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import StandardScaler
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

In [5]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    return ' '.join(tokens)

In [11]:
def vectorize_and_reduce_text(df, text_columns, vectorization_method='tfidf', 
                               reduction_method='pca', n_components=10):

    for col in text_columns:
        col_text = df[col].apply(preprocess_text)
        # combine all text columns
        if col == text_columns[0]:
            df['preprocessed_text'] = col_text
        else:
            df['preprocessed_text'] = df['preprocessed_text'] + ' ' + col_text
    
    # Vectorization
    if vectorization_method == 'tfidf':
        vectorizer = TfidfVectorizer(max_features=1000)
    else:  # count vectorization
        vectorizer = CountVectorizer(max_features=1000)
    
    # Fit and transform text to vector
    text_vectors = vectorizer.fit_transform(df['preprocessed_text'])
    
    # Dimensionality Reduction
    if reduction_method == 'pca':
        reducer = PCA(n_components=n_components)
        # Standardize first for PCA
        text_vectors_scaled = StandardScaler().fit_transform(text_vectors.toarray())
        reduced_vectors = reducer.fit_transform(text_vectors_scaled)
    else:  # Truncated SVD (works well with sparse matrices)
        reducer = TruncatedSVD(n_components=n_components)
        reduced_vectors = reducer.fit_transform(text_vectors)
    
    # Create DataFrame with reduced vectors
    reduced_df = pd.DataFrame(
        reduced_vectors, 
        columns=[f'{reduction_method}_text_component_{i+1}' for i in range(n_components)]
    )
    
    # Combine original DataFrame with reduced text features
    result_df = pd.concat([df, reduced_df], axis=1)
    
    return result_df, reducer, vectorizer

In [12]:
job_df_vector,_,_ = vectorize_and_reduce_text(job_df, ['skill_name','industry_name','company_name'], n_components=5)

In [13]:
job_df_vector.head(5)

,job_id,company_id,skill_abr,skill_name,industry_id,industry_name,company_size,country,employee_count,follower_count,...,price (GBP),preprocessed_skill_name,preprocessed_industry_name,preprocessed_company_name,preprocessed_text,pca_text_component_1,pca_text_component_2,pca_text_component_3,pca_text_component_4,pca_text_component_5
0,921716,2774458.0,MRKT,Marketing,44,Real Estate,2,US,402,2351,...,0.0,marketing,real estate,corcoran sawyer smith,marketing real estate corcoran sawyer smith,-0.368512,0.227145,-0.454321,0.115129,-0.182726
1,921716,2774458.0,SALE,Sales,44,Real Estate,2,US,402,2351,...,0.0,sales,real estate,corcoran sawyer smith,sales real estate corcoran sawyer smith,-0.379216,0.213018,-0.443585,0.106975,-0.189599
2,10998357,64896719.0,MNFC,Manufacturing,32,Restaurants,1,US,15,40,...,0.0,manufacturing,restaurants,national exemplar,manufacturing restaurants national exemplar,-1.019100,3.282573,2.395577,0.102374,0.123930
3,10998357,64896719.0,MGMT,Management,32,Restaurants,1,US,15,40,...,0.0,management,restaurants,national exemplar,management restaurants national exemplar,-1.000771,3.228462,2.445832,0.061840,0.209240
4,3899530060,64896719.0,MNFC,Manufacturing,32,Restaurants,1,US,15,40,...,0.0,manufacturing,restaurants,national exemplar,manufacturing restaurants national exemplar,-1.019100,3.282573,2.395577,0.102374,0.123930


In [14]:
job_df_vector.to_csv('Projectdata/job_data_vectorized.csv', index=False)